In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import mplleaflet as ml
import requests

In [ ]:
Flying from Miami to DC, we decided to play with the in flight wifi.

American Airlines gives you the following json interface, we are going to try to put it on a map.

In [ ]:
At first we had trouble getting connected, the solution was to manually insert the DNS entry 172.19.134.2

In [8]:
%%bash
curl http://airborne.gogoinflight.com/abp/ws/absServices/statusTray | python -m json.tool

{
    "Response": {
        "flightInfo": {
            "HSpeed": 239.3246,
            "VSpeed": -0.56896,
            "abpVersion": "4.0.4",
            "acpuVersion": "6.4.0",
            "airlineCode": "AAL",
            "airlineCodeIata": "AA",
            "airlineName": "American Airlines",
            "altitude": 9443.923,
            "departureAirportCode": "KMIA",
            "departureAirportCodeIata": "MIA",
            "destination": "DCA",
            "destinationAirportCode": "KDCA",
            "destinationAirportCodeIata": "DCA",
            "expectedArrival": "2016-06-06T13:28:00Z",
            "flightNumberAlpha": "AAL",
            "flightNumberInfo": "AAL1359",
            "flightNumberNumeric": 1359,
            "latitude": 28.011,
            "localTime": "2016-06-06T11:36:03.548Z0:0",
            "longitude": -81.1471,
            "origin": "MIA",
            "tailNumber": "N905AN",
            "utcTime": "2016-06-06T11:36:03.548Z",
            "videoService": fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   978  100   978    0     0   8676      0 --:--:-- --:--:-- --:--:--  8732
